<a href="https://colab.research.google.com/github/j15975383/DSMA/blob/main/DSMAv4_Goemotion_import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/monologg/GoEmotions-pytorch.git
!gdown --id "1j_bwko_aAYGs5W0wqY7cvIFqPIkNcQv7" --output "test_set_v2.csv"

Cloning into 'GoEmotions-pytorch'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 45 (delta 2), reused 6 (delta 2), pack-reused 35
Unpacking objects: 100% (45/45), done.
Downloading...
From: https://drive.google.com/uc?id=1j_bwko_aAYGs5W0wqY7cvIFqPIkNcQv7
To: /content/test_set_v2.csv
100% 70.0k/70.0k [00:00<00:00, 4.75MB/s]


In [ ]:
%cd /content/GoEmotions-pytorch

/content/GoEmotions-pytorch


In [ ]:
!pip install torch #--force--reinstall
!pip install transformers #--force--reinstall
!pip install attrdict

In [ ]:
import transformers
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline
from pprint import pprint

tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-group")
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-group")

#goemotions = MultiLabelPipeline(
    #model=model,
    #tokenizer=tokenizer,
    #threshold=0.3
#)



In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


No GPU available, using the CPU instead.


In [ ]:
#just to test things out
text = ["I hate you Anakin","You were like a brother to me"]
tokens = tokenizer.batch_encode_plus(text,padding = True, return_token_type_ids=False)
print(tokens)
tokens['input_ids']
tokenizer.batch_decode(tokens['input_ids'])
#or we can use
#tokenizer.decode(tokens['input_ids'][0])
#to find the decoded first sentence

{'input_ids': [[101, 146, 4819, 1128, 9954, 4314, 102, 0, 0], [101, 1192, 1127, 1176, 170, 1711, 1106, 1143, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}


['[CLS] I hate you Anakin [SEP] [PAD] [PAD]',
 '[CLS] You were like a brother to me [SEP]']

In [ ]:
import pandas as pd

test = pd.read_csv('/content/test_set_v2.csv')

In [ ]:
test

,Unnamed: 0,text,label
0,0,Xbox one X player along with series X player ...,1
1,1,>Yep all they had to do was just reskin BF5 wi...,1
2,2,Or 6 Jeep’s,0
3,3,I did and four out of my five friends who play...,1
4,4,Dont know about this argument because IMO ever...,2
...,...,...,...
195,195,"Me too, and now I’m sad.",1
196,196,I liked it but it wasn't quite bf. I hope this...,1
197,197,This might be a stupid question. (I played may...,2
198,198,They actually removed suppression lmao what th...,1


In [ ]:
max_seq_len = 50

In [ ]:
tokens_test = tokenizer.batch_encode_plus(
    test["text"].tolist(),
    max_length = max_seq_len,
    padding = True,
    truncation=True,
    return_token_type_ids=False
)

In [ ]:
test['label'] = test['label'].apply(lambda x: int(x))

In [ ]:
import torch
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test['label'].tolist())

In [ ]:
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds[0].numpy()

AttributeError: ignored

In [ ]:
preds = preds[0].numpy()

In [ ]:
import numpy as np
preds = np.argmax(preds, axis = 1)


AxisError: ignored

In [ ]:
#importing a fine-tuned model by monologg
from sklearn.metrics import classification_report
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.26      0.13      0.17        54
           1       0.72      0.30      0.43        76
           2       0.15      0.39      0.21        31
           3       0.38      0.59      0.46        39

    accuracy                           0.33       200
   macro avg       0.38      0.35      0.32       200
weighted avg       0.44      0.33      0.33       200



In [ ]:
df = pd.read_csv("/content/test_set.csv")

In [ ]:
test_set_np = df.copy()

In [ ]:
test_set_np.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  200 non-null    int64 
 1   text        200 non-null    object
 2   label       200 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ KB


In [ ]:
tokens_test = tokenizer.batch_encode_plus(
    test_set_np["text"].tolist(),
    max_length = max_seq_len,
    padding = True,
    truncation=True,
    return_token_type_ids=False
)

In [ ]:
labels = test_set_np['label'] 

In [ ]:
def change(x):
  if x == 0:
    return 1
  elif x ==1:
    return 3

test_set_np['label'] = test_set_np['label'].apply(lambda x: change(x))  

In [ ]:
test_set_np['label']

0      3
1      1
2      1
3      1
4      3
      ..
195    1
196    3
197    1
198    3
199    1
Name: label, Length: 200, dtype: int64

In [ ]:
test_set_np['label'] = test_set_np['label'].apply(lambda x: int(x))

In [ ]:
test_set_np['label'].value_counts(normalize = True)

1    0.625
3    0.375
Name: label, dtype: float64

In [ ]:
import torch
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_set_np['label'].tolist())

In [ ]:
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds[0].numpy()

AttributeError: ignored

In [ ]:
preds = preds[0].numpy()

In [ ]:
import numpy as np
preds = np.argmax(preds, axis = 1)


In [ ]:
#using multilabeled model on test_set that only has N and P
from sklearn.metrics import classification_report
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.87      0.26      0.40       125
           2       0.00      0.00      0.00         0
           3       0.60      0.52      0.56        75

    accuracy                           0.36       200
   macro avg       0.37      0.20      0.24       200
weighted avg       0.77      0.36      0.46       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(type(preds))

<class 'numpy.ndarray'>


In [ ]:
pddf = pd.DataFrame(preds)
pddf

,0
0,2
1,2
2,2
3,0
4,2
...,...
195,1
196,3
197,1
198,3


In [ ]:
pddf[0].value_counts(normalize = True)

2    0.345
3    0.325
1    0.190
0    0.140
Name: 0, dtype: float64